## Analise para redução do volume de municipios trabalhados

In [21]:
import pandas as pd

In [22]:
# --- 1. Carregar PIB dos municípios ---
pib_df = pd.read_csv('tabela5938.csv', sep=',', encoding='utf-8', skiprows=4)
pib_df.columns = [ 'codigo_municipio', 'municipio', 'pib_total']
pib_df

,codigo_municipio,municipio,pib_total
0,1100023,Ariquemes (RO),3209761.0
1,1100031,Cabixi (RO),238412.0
2,1100049,Cacoal (RO),2792383.0
3,1100056,Cerejeiras (RO),743037.0
4,1100064,Colorado do Oeste (RO),424807.0
...,...,...,...
5576,0,Zero resultante de um cálculo ou arredondament...,NaN
5577,X,Valor inibido para não identificar o informant...,NaN
5578,..,Valor não se aplica.\nEx: Não se pode obter o ...,NaN
5579,...,Valor não disponível.\nEx: A produção de feijã...,NaN


In [23]:
# --- 2. Carregar População dos municípios ---
pop_df = pd.read_csv('tabela6579.csv', sep=',', encoding='utf-8', skiprows=4)
pop_df.columns = ['municipio', 'populacao']
pop_df

,municipio,populacao
0,Ariquemes (RO),108573
1,Cabixi (RO),5690
2,Cacoal (RO),97637
3,Cerejeiras (RO),16975
4,Colorado do Oeste (RO),16588
...,...,...
5714,0,Zero resultante de um cálculo ou arredondament...
5715,X,Valor inibido para não identificar o informant...
5716,..,Valor não se aplica.\nEx: Não se pode obter o ...
5717,...,Valor não disponível.\nEx: A produção de feijã...


### Tratamento do DataFrame

In [24]:
def limpar_e_converter(df, valor_col):
    # Garante que os valores sejam string
    df[valor_col] = df[valor_col].astype(str)

    # Mantém apenas linhas com números (removendo tudo que não for número, ponto ou vírgula)
    df = df[df[valor_col].str.match(r'^[\d\.,]+$', na=False)].copy()

    # Converte o valor para float (milhares com ponto e decimal com vírgula)
    df[valor_col] = (
        df[valor_col]
        .str.replace('.', '', regex=False)  # remove separador de milhar
        .str.replace(',', '.', regex=False)  # troca vírgula decimal por ponto
        .astype(float)
    )
    return df



In [25]:
# --- PIB ---
pib_df = limpar_e_converter(pib_df, 'pib_total')

# --- População ---
pop_df = limpar_e_converter(pop_df, 'populacao')
pop_df

,municipio,populacao
0,Ariquemes (RO),108573.0
1,Cabixi (RO),5690.0
2,Cacoal (RO),97637.0
3,Cerejeiras (RO),16975.0
4,Colorado do Oeste (RO),16588.0
...,...,...
5564,Vianópolis (GO),15476.0
5565,Vicentinópolis (GO),9077.0
5566,Vila Boa (GO),4185.0
5567,Vila Propício (GO),5982.0


In [26]:
print("Duplicados em pop_df:", pop_df['municipio'].duplicated().sum())
print("Duplicados em pib_df:", pib_df['municipio'].duplicated().sum())

Duplicados em pop_df: 0
Duplicados em pib_df: 0


In [27]:
# --- Juntar e filtrar ---
df = pd.merge(pop_df, pib_df, on='municipio')

# Filtros
df_filtrado = df[(df['populacao'] > 50000) & (df['pib_total'] > 1_000_000)]

In [28]:
print("Municípios com população > 50 mil e PIB > R$ 1 bilhão:")
df_filtrado = df_filtrado.sort_values(by='codigo_municipio', ascending=False)
df_filtrado.head(10)

Municípios com população > 50 mil e PIB > R$ 1 bilhão:


,municipio,populacao,codigo_municipio,pib_total
5568,Brasília (DF),2982818.0,5300108,2.869438e+09
5562,Valparaíso de Goiás (GO),213506.0,5221858,2.964512e+07
5554,Trindade (GO),150858.0,5221403,2.666621e+07
5545,Senador Canedo (GO),169849.0,5220454,4.765089e+07
5534,Santo Antônio do Descoberto (GO),74614.0,5219753,8.256180e+06
5519,Rio Verde (GO),238025.0,5218805,1.630627e+08
5508,Planaltina (GO),110619.0,5217609,1.453001e+07
5488,Novo Gama (GO),107092.0,5215231,1.154068e+07
5471,Morrinhos (GO),53640.0,5213806,1.857437e+07
5465,Mineiros (GO),73781.0,5213103,3.288692e+07


In [29]:
# salvando como TXT (nome, código)
with open('municipios_filtrados.txt', 'w', encoding='utf-8') as f:
    for _, row in df_filtrado.iterrows():
        f.write(f"{row['municipio']},{row['codigo_municipio']}\n")